In [37]:
!pip install -r requirements.txt

In [38]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from datasets import load_dataset

### Prétraitement des Données

* Nous commençons par charger les données depuis un fichier CSV et afficher les informations sur les sentiments pour comprendre la structure des données.
* Ensuite, nous définissons une fonction de prétraitement pour nettoyer les tweets en enlevant les caractères spéciaux, les URLs, les mentions et les hashtags, et en supprimant les stopwords.


In [39]:

#dataset = pd.read_csv("C:/Users/flavi/Downloads/fifa_world_cup_2022_tweets.csv")

# Charger le dataset
dataset = load_dataset("Tirendaz/fifa-world-cup-2022-tweets")

tweets = dataset['train']['Tweet']
sentiments = dataset['train']['Sentiment']
# Créer un DataFrame avec les deux listes
dataset = pd.DataFrame({
    'Tweet': tweets,
    'Sentiment': sentiments
})
dataset

,Tweet,Sentiment
0,What are we drinking today @TucanTribe \n@MadB...,neutral
1,Amazing @CanadaSoccerEN #WorldCup2022 launch ...,positive
2,Worth reading while watching #WorldCup2022 htt...,positive
3,Golden Maknae shinning bright\n\nhttps://t.co/...,positive
4,"If the BBC cares so much about human rights, h...",negative
...,...,...
22519,Here We go World cup 2022 #WorldCup2022,positive
22520,Anderlecht confirms former Viborg FF's Jesper ...,neutral
22521,Great thread to read before the start of #Worl...,positive
22522,Raphinha wants Brazil to be united at the #Wor...,positive


In [40]:
# Télécharger les stopwords de NLTK si nécessaire
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Fonction pour prétraiter les tweets
def preprocess_tweet(tweet):
    # Remplacer les retours à la ligne par des espaces
    tweet = re.sub(r'\n', ' ', tweet)
    # Remplacer les retours chariots par des espaces
    tweet = re.sub(r'\r', ' ', tweet)
    # Enlever les URLs
    tweet = re.sub(r'http\S+|www\S+', '', tweet)
    # Enlever les mentions
    tweet = re.sub(r'@\w+', '', tweet)
    # Enlever les hashtags
    tweet = re.sub(r'#\w+', '', tweet)
    # Enlever les caractères spéciaux en gardant seulement les alphanumériques et les espaces
    tweet = ''.join([char for char in tweet if char.isalnum() or char.isspace()])
    # Convertir en minuscules
    tweet = tweet.lower()
    # Enlever les stopwords
    tweet = ' '.join([word for word in tweet.split() if word not in stop_words])
    return tweet

# Appliquer le prétraitement à chaque tweet
tweets_df = dataset['Tweet'].apply(preprocess_tweet)

# Afficher les tweets prétraités
print(tweets_df)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cypri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0                                           drinking today
1        amazing launch video shows much face canada me...
2                                   worth reading watching
3                            golden maknae shinning bright
4        bbc cares much human rights homosexual rights ...
                               ...                        
22519                                    go world cup 2022
22520    anderlecht confirms former viborg ffs jesper f...
22521                              great thread read start
22522                         raphinha wants brazil united
22523    buy sot pinksale confused buy tokens pinksale ...
Name: Tweet, Length: 22524, dtype: object


### Tokenisation et Préparation des Séquences

* Nous utilisons Tokenizer de Keras pour transformer les tweets en séquences de tokens. La taille du vocabulaire est augmentée à 10000 mots.
* Les séquences sont ensuite remplies (padding) à une longueur fixe de 120 tokens.

In [41]:
# Tokenisation : Transformer les tweets en séquences de tokens
tokenizer = Tokenizer(num_words=10000)  
tokenizer.fit_on_texts(tweets_df)
X = tokenizer.texts_to_sequences(tweets_df)

# Padding des séquences : Normaliser les séquences à une longueur fixe
X = pad_sequences(X, maxlen=60)  # Augmentation de la longueur maximale des séquences

# Convertir les labels de sentiment en valeurs numériques
dataset['Sentiment'] = dataset['Sentiment'].map({'negative': 0, 'neutral': 1, 'positive': 2})
y = dataset['Sentiment'].values

In [42]:
print(dataset['Sentiment'].unique())

[1 2 0]


### Définition et Entraînement du Modèle LSTM

* Nous définissons un modèle LSTM bidirectionnel avec des couches de dropout pour régulariser les données et éviter le surapprentissage.
* Le modèle est compilé avec la perte sparse_categorical_crossentropy et l'optimiseur Adam.
* Nous entraînons le modèle sur les données d'entraînement et validons sur les données de test.

In [43]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, SpatialDropout1D, Bidirectional
from tensorflow.keras.optimizers import Adam

# Définition de la classe pour le modèle LSTM personnalisé
class CustomLSTMModel:
    def __init__(self, vocab_size, embedding_dim, input_length, hidden_dim, output_size):
        # Initialiser le modèle séquentiel
        self.model = Sequential()
        # Ajouter une couche d'embedding
        self.model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=input_length))
        # Ajouter un SpatialDropout pour régulariser les embeddings
        self.model.add(SpatialDropout1D(0.3)) 
        # Ajouter une couche LSTM bidirectionnelle avec dropout
        self.model.add(Bidirectional(LSTM(hidden_dim, return_sequences=True, dropout=0.3, recurrent_dropout=0.3)))
        # Ajouter une couche LSTM avec dropout
        self.model.add(LSTM(hidden_dim, dropout=0.3, recurrent_dropout=0.3))
        # Ajouter une couche dense avec activation softmax pour la classification
        self.model.add(Dense(output_size, activation='softmax'))
        
        # Compiler le modèle avec la perte 'sparse_categorical_crossentropy' et l'optimiseur Adam
        self.model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

    # Fonction pour entraîner le modèle
    def fit(self, X_train, y_train, validation_data, epochs=10, batch_size=64):  # Augmentation des époques
        history = self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=validation_data, verbose=1)
        return history

    # Fonction pour évaluer le modèle
    def evaluate(self, X_test, y_test):
        loss, accuracy = self.model.evaluate(X_test, y_test, verbose=1)
        return loss, accuracy

    # Fonction pour prédire les classes des nouvelles données
    def predict(self, X_test):
        return self.model.predict(X_test)

# Assurer que les données sont sous forme de tableau numpy
X = np.array(X)
y = np.array(y)

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Initialiser et entraîner le modèle LSTM personnalisé

In [44]:

# Initialiser et entraîner le modèle LSTM personnalisé

# Taille du vocabulaire ajustée
vocab_size = 10000 

# Dimension des embeddings 
embedding_dim = 50 #128 #50 

# Longueur maximale des séquences
input_length = 60 #120 #60

# Nombre de neurones dans la couche cachée LSTM
hidden_dim = 32 #128 #32

# Pour trois classes : négatif, neutre, positif
output_size = 3  

# Nombre d'époques d'entraînement
epochs= 4 #10 #4

# Taille des lots
batch_size = 16 #64 #16


custom_lstm_model = CustomLSTMModel(vocab_size, embedding_dim, input_length, hidden_dim, output_size)
custom_lstm_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size)


Epoch 1/4
1127/1127 [==============================] - 78s 62ms/step - loss: 0.8102 - accuracy: 0.6170 - val_loss: 0.6774 - val_accuracy: 0.7070
Epoch 2/4
1127/1127 [==============================] - 74s 66ms/step - loss: 0.5661 - accuracy: 0.7646 - val_loss: 0.6556 - val_accuracy: 0.7154
Epoch 3/4
1127/1127 [==============================] - 73s 65ms/step - loss: 0.4729 - accuracy: 0.8099 - val_loss: 0.7039 - val_accuracy: 0.7103
Epoch 4/4
1127/1127 [==============================] - 71s 63ms/step - loss: 0.4178 - accuracy: 0.8315 - val_loss: 0.7323 - val_accuracy: 0.7072


Interprétation :

* La perte d'entraînement diminue bien, atteignant 0.4187, et l'exactitude d'entraînement augmente à 83.17%.
* La perte de validation continue d'augmenter légèrement, atteignant 0.7109, ce qui renforce l'indication de surapprentissage.
* L'exactitude de validation reste stable à 70.74%, indiquant que malgré l'augmentation de la performance sur les données d'entraînement, le modèle ne s'améliore pas sur les données de validation.

### Évaluation et Prédiction

* Nous évaluons le modèle sur les données de test pour obtenir la perte et la précision.
* Nous prédisons les sentiments des tweets de test et affichons un rapport de classification pour évaluer les performances du modèle.
* Pour comparaison, nous entraînons également un modèle de régression logistique et comparons ses performances avec le modèle LSTM.

In [45]:
from sklearn.metrics import classification_report, accuracy_score

# Évaluer le modèle
loss, accuracy = custom_lstm_model.evaluate(X_test, y_test)
print(f'Test Loss: {loss} | Test Accuracy: {accuracy}')

# Prédire et imprimer le rapport de classification
y_pred = np.argmax(custom_lstm_model.predict(X_test), axis=-1)
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=['negative', 'neutral', 'positive']))

# Modèle de régression logistique pour comparaison
from sklearn.linear_model import LogisticRegression

# Entraîner le modèle de régression logistique
logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X_train, y_train)

# Prédire et évaluer
y_pred_logistic = logistic_model.predict(X_test)
print("----------------------------------------------------------------------------")
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_logistic))
print(classification_report(y_test, y_pred_logistic, target_names=['negative', 'neutral', 'positive']))


141/141 [==============================] - 2s 11ms/step - loss: 0.7323 - accuracy: 0.7072
Test Loss: 0.7323295474052429 | Test Accuracy: 0.7072141766548157
141/141 [==============================] - 2s 11ms/step
Classification Report:
              precision    recall  f1-score   support

    negative       0.73      0.72      0.72      1149
     neutral       0.67      0.62      0.65      1648
    positive       0.72      0.78      0.75      1708

    accuracy                           0.71      4505
   macro avg       0.71      0.71      0.71      4505
weighted avg       0.71      0.71      0.71      4505

----------------------------------------------------------------------------
Logistic Regression Accuracy: 0.3908990011098779
              precision    recall  f1-score   support

    negative       0.27      0.03      0.05      1149
     neutral       0.37      0.36      0.37      1648
    positive       0.41      0.67      0.50      1708

    accuracy                           0

c:\Users\cypri\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Interprétation

<u>Modèle LSTM</u>:

* L'exactitude de test de 70.74% est cohérente avec l'exactitude de validation observée pendant l'entraînement, suggérant que le modèle généralise bien aux données de test.
  * Test Loss (Perte de Test) : 71.09%
  * Test Accuracy (Exactitude de Test) : 70.74%

* La classe "positive" a la meilleure précision, tandis que la classe "neutral" a le meilleur rappel.

<u>Régression logistique</u>:

* L'exactitude du modèle de régression logistique est bien inférieure à celle du modèle LSTM, indiquant que la régression logistique a du mal à capturer les relations complexes dans les données de tweets.

* La performance de la régression logistique est particulièrement faible pour la classe "negative", avec un rappel de seulement 3%. Cela suggère que le modèle de régression logistique ne parvient pas à capturer les caractéristiques des tweets négatifs.


L'exactitude du modèle de régression logistique est bien inférieure à celle du modèle LSTM, indiquant que la régression logistique a du mal à capturer les relations complexes dans les données de tweets.
* Test Accuracy Régression Logistique : 39.09%
* Test Accuracy Modèle LSTM : 70.74%

<u>ConvergenceWarning </u>:

L'avertissement indique que l'optimiseur L-BFGS a atteint le nombre maximum d'itérations avant de converger. Pour résoudre ce problème, il est possible :

* D'augmenter le nombre d'itérations (max_iter).
* De prétraiter les données en les normalisant ou en les standardisant.